# Autoencoders
In this notebook we will develop a simple autoencoder in pytorch.<br>
Autoencoders fall under the unsupervised family, and thier purpose is to learn an encoding of the 
data, where the immediate use case is dimensionality reduction.<br>
Autencoders where first purposed by [Rumelhart](https://apps.dtic.mil/docs/citations/ADA164453) under Hinton supervision. <br><br>

## Intution
The best way to start explaining an autoencoder is by using the following visualization: 
![alt text](img/cells.png "Title")

The first thing we notice is that the input and output layer have the same size. <br>
This is because our "goal" is that the input and the output will be exactly the same! To put antoher way, if we input $x$ we want to get $x$ as the output.<br>
The second thing we notice is that we have a hidden layer with a lower dimensionality than the input and output layer. This is where our "encoding" hides, we want to shrink the input, and still be able to recover it.

## Theory
Mathmetically you can define an autoencoder operating on real values using the following tuple :
- $n, p$ are integers. (in the above visualiztion $n=8$ and $p=3$)
- $f$ a function $f:\mathbb{R}^n\rightarrow \mathbb{R}^p$ 
- $g$ a function $g:\mathbb{R}^p\rightarrow \mathbb{R}^n$ <br>
Under this definition, given an input $x\in \mathbb{R}^n$ an autoencoder output is simply the composition $g\circ f(x)$ <br>

Let's extend this to formulate the trainig.
- $F$ the family of functions  $f:\mathbb{R}^n\rightarrow \mathbb{R}^p$ 
- $G$ the family of functions $g:\mathbb{R}^p\rightarrow \mathbb{R}^n$ <br>
- $X=\{x_1, x_2, \ldots, x_m\}$ is our training set. $\forall i ~ x_i \in \mathbb{R}^n$
- $L$ will be a loss measure $ L : \mathbb{R}^n \times \mathbb{R}^n \rightarrow \mathbb{R}$
